# Connecting to a port in the container

Another common use case is that we start some kind of long-running process in the container, and talk to it through a port. That process could be a Jupyter Notebook, for example. 

For demonstration purposes, we will use Python's in-built web server. To run it from the host, we could use

In [7]:
# NBVAL_SKIP
!python -m http.server


Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
^C

Keyboard interrupt received, exiting.


This program will show the contents of the current file system in a webbrowser interface at port 8000 of this machine. So typically at one or some of these links http://127.0.0.1:8000 or http://localhost:8000 or http://0.0.0.0:8000).

(If you have executed the above cell by pressing SHIFT+RETURN, you need to interrupt the http.server process to get the control back in the notebook. This can be done by choosing from the menu "Kernel" -> "Interrupt".)

We will now create a container and run this server inside the container. We like to use a webbrowser on the host machine to inspect the files.

First, we create the Dockerfile:

In [4]:
%%file Dockerfile
FROM ubuntu:16.04

RUN apt-get update
RUN apt-get install python3

CMD python -m http.server /

Overwriting Dockerfile


The last line starts the `http.server` when the container is run.

In [5]:
!docker build -t portdemo .

Sending build context to Docker daemon 73.73 kB
Step 1/3 : FROM ubuntu:16.04
 ---> 0ef2e08ed3fa
Step 2/3 : RUN apt-get update
 ---> Using cache
 ---> 64d0cc85e4a4
Step 3/3 : RUN apt-get install python3
 ---> Running in b33fd2248a88
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  dh-python file libexpat1 libmagic1 libmpdec2 libpython3-stdlib
  libpython3.5-minimal libpython3.5-stdlib libsqlite3-0 libssl1.0.0
  mime-support python3-minimal python3.5 python3.5-minimal
Suggested packages:
  libdpkg-perl python3-doc python3-tk python3-venv python3.5-venv
  python3.5-doc binutils binfmt-support
The following NEW packages will be installed:
  dh-python file libexpat1 libmagic1 libmpdec2 libpython3-stdlib
  libpython3.5-minimal libpython3.5-stdlib libsqlite3-0 libssl1.0.0
  mime-support python3 python3-minimal python3.5 python3.5-minimal
0 upgraded, 15 newly installed, 0 to remove and 7 not upgraded.
Need 

We now need to export the port 8000 in the container. We can do this using:

In [8]:
!docker run -p 8000:8123 portdemo

Unable to find image 'portdemo:latest' locally
docker: Error response from daemon: repository portdemo not found: does not exist or no pull access.
See 'docker run --help'.


The numbers following reflect the internal port (8000) that should be connected to the port (8123) on the host system.

Once the above command is executing, we should be able to browse the file system in the container by going to the link http://localhost:8123 (or http://127.0.0.1:8123 or http://0.0.0.0:8123) on the host system. 

We could have mapped port 8000 in the container to port 8000 on the host as well (`-p 8000:8000).

## Jupyter Notebook

A common application of exposing ports is to install computational or data analysis software inside the container, and to control it from a jupyter notebook running inside the container, but to use a webbrowser from the host system to interact with the notebook. In that case, the above example of exposing the port is in principle the right way to go, too. However, as this is a common usecase, there are a number of prepared Dockerfiles to install the notebook inside the container available at https://github.com/jupyter/docker-stacks, so tat one can start the Dockerfile with `FROM jupyter/...`, (instead of `FROM ubuntu/...`) and in this way build on the Dockerfiles that the Jupyter team provides already.